In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/ilamosin/abTest_pet/app_logs.csv', parse_dates=['date'])
df.head(5)

,date,user_id,exp_group,session_id,session_length,device,order_cnt,price,quantity_cnt
0,2022-06-21,1727468184902936832,0,16557919071727468184902936832,1752,mobile,0,0.0,0
1,2022-06-21,3556219827372626733,0,16558186423556219827372626733,86,mobile,0,0.0,0
2,2022-06-21,2136855388003761026,0,16558402842136855388003761026,222,mobile,0,0.0,0
3,2022-06-21,4593970126020001597,0,16558003654593970126020001597,400,mobile,0,0.0,0
4,2022-06-21,3962568860806737949,1,16558324233962568860806737949,1300,mobile,0,0.0,0


## EDA (exploratory data analysis)

We released new interface feature on all platforms and want to know is it helpful for our users and is there profit for our product and business.

date - дата 

user_id - айди пользователя

exp_group - группа (тестовая/контрольная)

session_id - айди сессии 

session_length - длинна сессии

device - устройство

order_cnt - количество заказов

price - суммарная стоимость

quantity_cnt - количество заказанных позиций?

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269142 entries, 0 to 269141
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            269142 non-null  datetime64[ns]
 1   user_id         269142 non-null  int64         
 2   exp_group       269142 non-null  int64         
 3   session_id      269142 non-null  object        
 4   session_length  269142 non-null  int64         
 5   device          269142 non-null  object        
 6   order_cnt       269142 non-null  int64         
 7   price           269142 non-null  float64       
 8   quantity_cnt    269142 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 18.5+ MB


In [4]:
print(f'Devices which users use: {df.device.unique()}')
print(f'Unique users count: {df.user_id.nunique()}')
print(f'Unique session count: {df.session_id.nunique()}')
print(f'Max date: {df.date.max()}')
print(f'Min date: {df.date.min()}')
print(f'Diff between count session_id and unique session_id : {df.session_id.count()-df.session_id.nunique()}')

Devices which users use: ['mobile' 'desktop' 'tablet']
Unique users count: 68287
Unique session count: 267929
Max date: 2022-07-13 00:00:00
Min date: 2022-06-21 00:00:00
Diff between count session_id and unique session_id : 1213


We have diff between count session_id and unique session_id. Lets see reason

In [96]:
df\
    .groupby(['user_id'])\
    .agg({'session_id':'count'})\
    .rename(columns={'session_id':'count_s'})\
    .query('count_s>1')\
    .sort_values('count_s', ascending=False)\
    .reset_index()

,user_id,count_s
0,7377259055888388543,174
1,2643534565406623178,98
2,5649845706405562462,76
3,7460365642247924721,71
4,7369859312574859099,69
...,...,...
43618,6467165119456300660,2
43619,6468351909120185336,2
43620,6469145490238627906,2
43621,2498253306598377956,2


In [137]:
u_1 = df[df['user_id']==6087415308146188769]
u_1['session_id'] = u_1['session_id'].astype(str).astype(float)
u_1.sort_values('session_id')

/var/folders/gs/p9gsqz5s7ks_j19rsq__zt840000gn/T/ipykernel_27750/787111243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u_1['session_id'] = u_1['session_id'].astype(str).astype(float)


,date,user_id,exp_group,session_id,session_length,device,order_cnt,price,quantity_cnt
21538,2022-06-22,6087415308146188769,1,1.655919e+28,1490,mobile,0,0.0,0
19002,2022-06-22,6087415308146188769,1,1.655921e+28,444,mobile,0,0.0,0
123840,2022-06-30,6087415308146188769,1,1.656620e+28,953,mobile,0,0.0,0
140215,2022-07-02,6087415308146188769,1,1.656766e+28,321,mobile,0,0.0,0
140183,2022-07-02,6087415308146188769,1,1.656773e+28,1516,mobile,0,0.0,0
143798,2022-07-02,6087415308146188769,1,1.656794e+28,1149,mobile,0,0.0,0
149475,2022-07-03,6087415308146188769,1,1.656794e+28,895,mobile,0,0.0,0
149171,2022-07-03,6087415308146188769,1,1.656825e+28,567,mobile,0,0.0,0
153690,2022-07-03,6087415308146188769,1,1.656834e+28,16,mobile,0,0.0,0
153601,2022-07-03,6087415308146188769,1,1.656854e+28,1003,mobile,0,0.0,0


In [138]:
1.656794e+28==1.656794e+28

True

In [162]:
1149+895

2044

In [140]:
u_2 = df[df['user_id']==6178250370357987265]
u_2['session_id'] = u_2['session_id'].astype(str).astype(float)
u_2.sort_values('session_id')

/var/folders/gs/p9gsqz5s7ks_j19rsq__zt840000gn/T/ipykernel_27750/982550787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u_2['session_id'] = u_2['session_id'].astype(str).astype(float)


,date,user_id,exp_group,session_id,session_length,device,order_cnt,price,quantity_cnt
5161,2022-06-21,6178250370357987265,1,1.655845e+28,633,mobile,0,0.0,0
23781,2022-06-22,6178250370357987265,1,1.655845e+28,316,mobile,0,0.0,0
31597,2022-06-23,6178250370357987265,1,1.655959e+28,1683,mobile,0,0.0,0
48387,2022-06-24,6178250370357987265,1,1.656059e+28,577,mobile,0,0.0,0
51182,2022-06-24,6178250370357987265,1,1.656083e+28,225,mobile,0,0.0,0
171883,2022-07-05,6178250370357987265,1,1.656969e+28,1511,mobile,0,0.0,0
223502,2022-07-09,6178250370357987265,1,1.657349e+28,779,mobile,0,0.0,0
259888,2022-07-13,6178250370357987265,1,1.657737e+28,1865,mobile,0,0.0,0


In [141]:
1.655845e+28==1.655845e+28

True

In [159]:
633+316

949

The reason why we have duplicated session_id is that users can start sesion at 23.59 PM 23.04.2022 and finish as 00.30 24.04.2022 

I think, we have app where users make orders, something like marketplace. 
Users can make their orders from devices like mobile, desktop, tablet.
Signals which we have: session_length, order_cnt, price, quantity_cnt

Now lets aggregate signals to the metriсs, some of it - average check, asl(Average session length), arpu, arppu, count of session, Total number of orders

Using sql for this

conversion: 
conversion to payment 

users:
arppu = sum(price)/sum(user_id which make order)
arpu = sum(price)/sum(user_id) 
asl = ((sum sessions length)/(sum count ssessions))
count of session = count of seesion on one user

ratio metrics:
average check = sum(price)/count(price)
average number of orders in the check
percentage of sessions that ended in payment

## Aggregating signals to the metrics

In [9]:
con = sqlite3.connect('db')

In [10]:
df.to_sql('metrics', con, index=False, if_exists='replace')

269142

In [374]:
sql = '''
with cte as (
        SELECT
            DATE(date) as date,
            iif(date<'2022-06-30', 0, 1) as is_exp_period,
            exp_group,
            user_id,
            device,
            session_id,
            price,
            quantity_cnt,
            order_cnt,
            iif(price>0, 1, 0) as is_paid,
            sum(session_length) as session_length
        FROM 
            metrics
        GROUP BY
            exp_group,
            user_id,
            device,
            session_id
)
SELECT 
    is_exp_period,
    exp_group,
    user_id,
    device,
    is_paid,
    sum(price) as spendings,
    sum(quantity_cnt) as goods_cnt,
    sum(order_cnt) as order_cnt,
    count(session_id) as session_cnt,
    log(sum(session_length)) as total_session_log_length,
    count(iif (price>0, 1, NULL)) as count_payed_ses
FROM
    cte
GROUP BY
    is_exp_period,
    exp_group,
    user_id,
    device
'''

In [375]:
df_f = pd.read_sql(sql, con)

In [376]:
df[df['user_id']==6180726904333254161]

,date,user_id,exp_group,session_id,session_length,device,order_cnt,price,quantity_cnt
157890,2022-07-04,6180726904333254161,1,16569673066180726904333254161,1065,desktop,0,0.00,0
174354,2022-07-05,6180726904333254161,1,16570058026180726904333254161,777,desktop,0,0.00,0
175317,2022-07-05,6180726904333254161,1,16569673066180726904333254161,1958,desktop,0,0.00,0
180257,2022-07-06,6180726904333254161,1,16570841086180726904333254161,1819,desktop,1,45340.19,67


In [377]:
16569673066180726904333254161==16569673066180726904333254161

True

In [378]:
1065+1958

3023

In [379]:
df_f[df_f['user_id']==6180726904333254161]

,is_exp_period,exp_group,user_id,device,is_paid,spendings,goods_cnt,order_cnt,session_cnt,total_session_log_length,count_payed_ses
83337,1,1,6180726904333254161,desktop,0,45340.19,67,1,3,3.749659,1


In [380]:
df_f[df_f['user_id']==7377259055888388543]

,is_exp_period,exp_group,user_id,device,is_paid,spendings,goods_cnt,order_cnt,session_cnt,total_session_log_length,count_payed_ses
16691,0,0,7377259055888388543,desktop,0,0.0,0,0,87,4.458547,0
61625,1,0,7377259055888388543,desktop,0,0.0,0,0,87,4.588047,0


In [381]:
df_f[df_f['user_id']==2643534565406623178]

,is_exp_period,exp_group,user_id,device,is_paid,spendings,goods_cnt,order_cnt,session_cnt,total_session_log_length,count_payed_ses
6014,0,0,2643534565406623178,desktop,1,98980.63,78,33,36,4.357782,32
48753,1,0,2643534565406623178,desktop,1,460012.35,333,75,62,5.237883,47
